In [1]:
from math import sqrt
import copy
import  traceback
import shutil
import random

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image
from tqdm import tqdm

In [2]:
def load_scan(path):
    slices = [] #slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    for s in os.listdir(path):
        if not s.endswith('.dcm'):
            continue
        sl = pydicom.dcmread(os.path.join(path, s))
        try:
            sl_p = sl.pixel_array
        except AttributeError:
            traceback.print_exc()
            print(f'\tDelete {os.path.join(path, s)}')
            os.remove(os.path.join(path, s))
        else:
            slices.append(sl)
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices

In [3]:
lower_b, upper_b = -100, 500

## 1.阳性数据

In [ ]:
# 将dcm文件根据窗宽窗位转化为png图片
def generate_image(input_folder):
    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

# generate_image('/nfs3-p1/zsxm/dataset/2021-9-8/')
# generate_image('/nfs3-p1/zsxm/dataset/2021-9-13/')
# generate_image('/nfs3-p1/zsxm/dataset/2021-9-19/')
# generate_image('/nfs3-p1/zsxm/dataset/2021-9-28/')
generate_image('/nfs3-p2/zsxm/dataset/2021-10-19-imh/')
generate_image('/nfs3-p2/zsxm/dataset/2021-10-19-pau/')
generate_image('/nfs3-p2/zsxm/dataset/2021-10-19-aa/')

In [ ]:
# 将各个病例中的png图片文件夹统一移动到一起供yolov5检测
def move_together_for_detect(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-8/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-13/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-19/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-28/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-10-19-imh/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-10-19-pau/', '/nfs3-p1/zsxm/dataset/9_detect/')
move_together_for_detect('/nfs3-p2/zsxm/dataset/aorta_temp/', '/nfs3-p1/zsxm/dataset/9_detect/')

In [ ]:
# shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/2021-9-8', '/nfs3-p1/zsxm/dataset/9_detect/pred')
# shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/2021-9-13', '/nfs3-p1/zsxm/dataset/9_detect/pred')
# shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/2021-9-19', '/nfs3-p1/zsxm/dataset/9_detect/pred')
# shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/2021-9-28', '/nfs3-p1/zsxm/dataset/9_detect/pred')
shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/aorta_temp', '/nfs3-p1/zsxm/dataset/9_detect/pred')

In [ ]:
# 用在更换窗宽窗位的时候，因为已经经过yolov5的检测生成了标签，故直接移动图片到images文件夹下即可
def directly_move_to_image_label(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        image_path = os.path.join(dst_path, name, 'images')
        if os.path.exists(image_path):
            print(f"\tremove {image_path}")
            shutil.rmtree(image_path)

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), image_path)
        except:
            traceback.print_exc()
            
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')

In [ ]:
# 将移动到待测文件夹中的原图移动到images文件夹
def move_ori_image_to_sigle_folder(input_path, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    for patient in os.listdir(input_path):
        print(f'****Processing {patient}****')
        if not os.path.exists(os.path.join(dst_path, patient)):
            os.mkdir(os.path.join(dst_path, patient))
        image_path = os.path.join(dst_path, patient, 'images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for img in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, img)):
                shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
    
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-28/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/aorta_temp/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/imh2')

In [ ]:
# 将yolov5的检测结果（标签和带bounding box的图像）移动到相应文件夹
def move_detect_result_to_folder(input_path, output_path):
    for patient in os.listdir(input_path):
        print(f'Processing {patient}')
        image_path = os.path.join(output_path, patient, 'pred_images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for file in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, file)):
                shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
            elif os.path.isdir(os.path.join(input_path, patient, file)):
                if os.path.exists(os.path.join(output_path, patient, file)):
                    shutil.rmtree(os.path.join(output_path, patient, file))
                shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
                
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-28/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/aorta_temp/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/imh2')

In [ ]:
# 切出主动脉
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.2 < y < 0.8, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.2 < corr1[2] < 0.8, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.2 < corr2[2] < 0.8, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = int(width*w), int(height*h)
    w, h = max(w, h), max(w, h)
    return int(width*x-w/2), int(height*y-h/2), int(width*x+w/2+1), int(height*y+h/2+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.lower().split('-')
        for i, s in enumerate(pl):
            if i!=0 and s.startswith('im'):
                pl[i] = s.replace('im', '')
        crop_flag = True
        for i, s in enumerate(pl):
            if s != 's' and s != 'j':
                continue
            start, end = int(pl[i+1])-1, int(pl[i+2])
            for j in range(start, end):
                img = Image.open(os.path.join(image_path, f'{patient}_{j:04d}.png'))
                img = np.array(img)
                try:
                    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
                except:
                    traceback.print_exc()
                    crop_flag = False
                else:#if crop_flag:
                    crop = img[y1:y2, x1:x2]
                    crop = Image.fromarray(crop)
                    crop.save(os.path.join(crop_path, f'{patient}_{s}_{j:04d}.png'))
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)

epl1 = []
epl2 = []

crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/imh2', epl1)
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau', epl2)

In [ ]:
print(len(epl1))
print(epl1)
# print(len(epl2))
# print(epl2)

In [ ]:
# # 将切出的主动脉移动到一起
# def move_to_classify(input_root, output_path):
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
    
#     for patient in os.listdir(input_root):
#         if os.path.isfile(os.path.join(input_root, patient)):
#             continue
#         print(f'****Processing {patient}****')
#         for file in os.listdir(os.path.join(input_root, patient, 'crops')):
#             shutil.copy(os.path.join(input_root, patient, 'crops', file), output_path)

# move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/1')
# move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/1')

In [5]:
# 切出范围外冗余为3的主动脉
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.2 < y < 0.8, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.2 < corr1[2] < 0.8, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.2 < corr2[2] < 0.8, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = int(width*w), int(height*h)
    w, h = max(w, h), max(w, h)
    return int(width*x-w/2), int(height*y-h/2), int(width*x+w/2+1), int(height*y+h/2+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.lower().split('-')
        for i, s in enumerate(pl):
            if i!=0 and s.startswith('im'):
                pl[i] = s.replace('im', '')
        crop_flag = True
        for i, s in enumerate(pl):
            if s != 's' and s != 'j':
                continue
            start, end = int(pl[i+1])-1, int(pl[i+2])
            for j in range(start-3, end+3):
                try:
                    img = Image.open(os.path.join(image_path, f'{patient}_{j:04d}.png'))
                    img = np.array(img)
                    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
                except:
                    traceback.print_exc()
                    crop_flag = False
                else:#if crop_flag:
                    crop = img[y1:y2, x1:x2]
                    crop = Image.fromarray(crop)
                    if start <= j < end:
                        crop.save(os.path.join(crop_path, f'{patient}_{s}_{j:04d}.png'))
                    else:
                        crop.save(os.path.join(crop_path, f'{patient}_{s}_{j:04d}_n.png'))
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)

epl1 = []
epl2 = []
epl3 = []

crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive', epl1)
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/imh', epl2)
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau', epl3)

******Processing baochunsheng-J-20-82******
******Processing baojinan-J-24-87******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/baojinan-J-24-87/labels/baojinan-J-24-87_0076.txt'
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/baojinan-J-24-87/labels/baojinan-J-24-87_0088.txt'
Traceback (most recent call last):
  File "/

******Processing baoyin-S-Im9-22-J-Im9-68******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/baoyin-S-Im9-22-J-Im9-68/labels/baoyin-S-Im9-22-J-Im9-68_0065.txt'
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/baoyin-S-Im9-22-J-Im9-68/labels/baoyin-S-Im9-22-J-Im9-68_0066.txt'


******Processing baozuoshi-J-16-79******
******Processing caijilu-J-Im41-87******
******Processing caimeiju-J-Im21-81******
******Processing caiweiguang-J-Im35-152******
******Processing caiying-J-Im34-52******
******Processing changzhenxuan-J-15-55******
******Processing chenaijiao-J-26-41******
******Processing chenaijun-S-Im25-38-J-Im25-85******


...

******Processing zhengxiusheng-S-26-27******
******Processing zhongjianhong-J-29-63******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhengxiusheng-S-26-27_0022.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhengxiusheng-S-26-27_0023.txt


******Processing zhongzhongnan-J-22-37******
******Processing zhougensheng-S-39-42******
******Processing zhourongcheng-J-80-93******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhougensheng-S-39-42_0035.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/132599818.py", line 47, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "/tmp/ipykernel_37540/132599818.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhougensheng-S-39-42_0036.txt


******Processing zhouxincang-J-40-97******
******Processing zhujianping-J-31-34******


In [6]:
print(len(epl1))
print(epl1)
print(len(epl2))
print(epl2)
print(len(epl3))
print(epl3)

138
['baojinan-J-24-87', 'baoyin-S-Im9-22-J-Im9-68', 'chenaijun-S-Im25-38-J-Im25-85', 'chenazhao-S-Im17-30', 'chencaiying-J-19-79', 'chenfuliang-S-21-31-J-21-84', 'chenglongdeng-S-Im21-33-J-Im21-78', 'chengyonghui-J-17-81', 'chenhui-S-17-25-J-17-87', 'chenjiandong-S-19-28-J-19-91', 'chenlibo-S-Im39-57', 'chenlili-S-Im18-24-J-Im18-74', 'chenmimao-S-Im32-40', 'chensiqi-S-17-28-J-17-69', 'chenxuehua-S-Im17-27-J-Im17-35', 'dingliyun-J-Im19-57', 'dingzhiqun-S-Im21-29', 'dukun-S-20-22-J-20-23', 'fangchunfeng-J-16-48', 'fangtianming-S-Im15-30-J-Im15-65', 'fangyuxian-J-23-60', 'fangzhuxiang-J-18-61', 'fanxinan-S-Im16-36-J-Im16-30', 'fudehe-S-20-33-J-20-70', 'gongyongfa-S-Im18-34-J-Im18-87', 'guanxujun-S-Im25-35-J-Im25-87', 'guquankang-J-Im18-69', 'hexingwen-S-Im25-35-J-Im25-93', 'hongjinjun-S-Im24-28-J-Im24-83', 'hongzhimin-S-Im30-39-J-30-144', 'huajin-J-Im30-67', 'huangdebing-S-Im40-48-J-Im33-138', 'huanghai-S-Im19-29-J-Im19-80', 'huangwanman-S-15-30', 'huweibang-S-Im25-30-J-Im25-79', 'huxiao

## 2.阴性数据

In [ ]:
# 将dcm文件根据窗宽窗位转化为png图片
def generate_image(input_folder):
    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

generate_image('/nfs3-p2/zsxm/dataset/2021-9-17-negative/')
generate_image('/nfs3-p2/zsxm/dataset/2021-9-29-negative/')

In [ ]:
# 将各个病例中的png图片文件夹统一移动到一起供yolov5检测
def move_together_for_detect(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

move_together_for_detect('/nfs3-p2/zsxm/dataset/2021-9-17-negative', '/nfs3-p1/zsxm/dataset/9_detect/')
move_together_for_detect('/nfs3-p2/zsxm/dataset/2021-9-29-negative', '/nfs3-p1/zsxm/dataset/9_detect/')

In [ ]:
shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/2021-9-17-negative', '/nfs3-p1/zsxm/dataset/9_detect/pred')
shutil.move('/disk1/zsxm/pythonWorkspace/yolov5/runs/detect/2021-9-29-negative', '/nfs3-p1/zsxm/dataset/9_detect/pred')

In [ ]:
# 用在更换窗宽窗位的时候，因为已经经过yolov5的检测生成了标签，故直接移动图片到images文件夹下即可
def directly_move_to_image_label(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        image_path = os.path.join(dst_path, name, 'images')
        if os.path.exists(image_path):
            print(f"\tremove {image_path}")
            shutil.rmtree(image_path)

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), image_path)
        except:
            traceback.print_exc()
            
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

In [ ]:
# 将移动到待测文件夹中的原图移动到images文件夹
def move_ori_image_to_sigle_folder(input_path, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    for patient in os.listdir(input_path):
        print(f'****Processing {patient}****')
        if not os.path.exists(os.path.join(dst_path, patient)):
            os.mkdir(os.path.join(dst_path, patient))
        image_path = os.path.join(dst_path, patient, 'images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for img in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, img)):
                shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
    
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-29-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

In [ ]:
# 将yolov5的检测结果（标签和带bounding box的图像）移动到相应文件夹
def move_detect_result_to_folder(input_path, output_path):
    for patient in os.listdir(input_path):
        print(f'Processing {patient}')
        image_path = os.path.join(output_path, patient, 'pred_images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for file in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, file)):
                shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
            elif os.path.isdir(os.path.join(input_path, patient, file)):
                if os.path.exists(os.path.join(output_path, patient, file)):
                    shutil.rmtree(os.path.join(output_path, patient, file))
                shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
                
move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')
move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-29-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

In [9]:
# 切出主动脉,这里有问题啊，branch_end前后0.3的切片切不出来，建议以后更改为和下面一样的方案
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.2 < y < 0.8, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.2 < corr1[2] < 0.8, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.2 < corr2[2] < 0.8, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = int(width*w), int(height*h)
    w, h = max(w, h), max(w, h)
    return int(width*x-w/2), int(height*y-h/2), int(width*x+w/2+1), int(height*y+h/2+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.split('-')
        assert len(pl) == 5
        aorta_start, branch_start, branch_end, aorta_end = int(pl[1])-1, int(pl[2])-1, int(pl[3])-1, int(pl[4])-1
        crop_flag = True
        offset = branch_end - branch_start
        start, end = branch_start + int(0.1*offset), branch_end - int(0.2*offset)
        for i in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_s_{i:04d}.png'))
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png'))
        offset = aorta_end - branch_end
        start, end = branch_end + int(0.1*offset), aorta_end - int(0.2*offset)
        for i in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png')) 
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)
            
epl1 = []
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative', epl1)

******Processing baimeizhu-27-37-54-136******
******Processing baixiaoxu-22-31-52-147******
******Processing baojinjia-17-21-35-83******
******Processing baoxialin-23-34-57-158******
******Processing baoyanzu-15-20-31-87******
******Processing baoyanzu-21-25-42-93******
******Processing bianyinqiao-31-38-59-137******
******Processing bianzhaorong-17-22-38-92******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/3585996673.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/3585996673.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：bianzhaorong-17-22-38-92_0023.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/3585996673.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/3585996673.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：bianzhaorong-17-22-38-92_0031.txt


******Processing caijinfei-34-42-51-155******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/3585996673.py", line 67, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "/tmp/ipykernel_37540/3585996673.py", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caijinfei-34-42-51-155/labels/caijinfei-34-42-51-155_0115.txt'


******Processing caijingnan1-41-50-73-184******


...

Traceback (most recent call last):
  File "/tmp/ipykernel_37540/3585996673.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/3585996673.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：huangjuzhen-13-17-29-77_0024.txt


******Processing huanglongsheng-23-28-46-92******
******Processing huangweigen-24-32-62-138******
******Processing huangxiang-22-26-42-84******
******Processing huangxuena-17-22-27-75******
******Processing huangzhanzhang-25-33-46-134******
******Processing huangzhaoxun-29-41-62-152******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/3585996673.py", line 67, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "/tmp/ipykernel_37540/3585996673.py", line 4, in find_coordinate
    lines = f.readlines()
KeyboardInterrupt


******Processing huaweiqin-15-25-33-77******
******Processing hukaipin-29-39-73-159******
******Processing humingdong-18-22-29-85******
******Processing huxiangxin-19-24-34-86******
******Processing huxianhua-32-41-55-143******
******Processing huyongxian-18-25-38-88******


KeyboardInterrupt: 

In [12]:
print(len(epl1))
print(epl1)

347
['baojinjia-17-21-35-83', 'baoyanzu-15-20-31-87', 'baoyanzu-21-25-42-93', 'bianyinqiao-31-38-59-137', 'bianzhaorong-17-22-38-92', 'caijinfei-34-42-51-155', 'caijingnan1-41-50-73-184', 'caishiping-18-24-34-85', 'caiyouzhe-14-18-32-83', 'caoyuxia-16-21-35-87', 'caoyuying-18-22-33-79', 'chenbin-20-26-35-85', 'chenchong-22-26-36-86', 'chendonghong-12-17-27-75', 'chengenye-13-18-28-79', 'chengshaoping-19-28-45-132', 'chengyahui-17-22-29-82', 'chengzhiqiang-33-41-67-159', 'chenjian-26-30-44-89', 'chenjianhua-14-17-32-80', 'chenjianping-15-22-41-91', 'chenjingjing-38-47-67-184', 'chenjinmei-12-17-34-80', 'chenjunbing-22-29-34-89', 'chenlidong-18-23-32-88', 'chenliuer-23-32-43-140', 'chenmaojin-14-20-31-90', 'chenqiuying-13-17-30-78', 'chenshigen-19-25-34-83', 'chenweiqian-22-31-45-131', 'chenxiufen-17-21-34-82', 'chenyanli-20-23-34-80', 'chenyinfen-18-22-38-81', 'chenyonghu-24-32-48-141', 'daicaihu-22-27-34-85', 'denghuibin-15-19-35-84', 'dingbinghai-20-24-45-95', 'dongxingzhen-9-13-23-77

In [ ]:
# # 将切出的主动脉移动到一起
# def move_to_classify(input_root, output_path):
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
    
#     for patient in os.listdir(input_root):
#         if os.path.isfile(os.path.join(input_root, patient)):
#             continue
#         print(f'****Processing {patient}****')
#         for file in os.listdir(os.path.join(input_root, patient, 'crops')):
#             shutil.copy(os.path.join(input_root, patient, 'crops', file), output_path)
            
# move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/0/')

In [10]:
# 切出范围外冗余为3的主动脉
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.2 < y < 0.8, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.2 < corr1[2] < 0.8, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.2 < corr2[2] < 0.8, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = int(width*w), int(height*h)
    w, h = max(w, h), max(w, h)
    return int(width*x-w/2), int(height*y-h/2), int(width*x+w/2+1), int(height*y+h/2+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.split('-')
        assert len(pl) == 5
        aorta_start, branch_start, branch_end, aorta_end = int(pl[1])-1, int(pl[2])-1, int(pl[3])-1, int(pl[4])-1
        crop_flag = True
        offset = branch_end - branch_start
        start, end = branch_start + int(0.1*offset), branch_end - int(0.2*offset)
        for i in range(start-3, end+3):
            try:
                img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
                img = np.array(img)
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                if start <= i < end:
                    crop.save(os.path.join(crop_path, f'{patient}_s_{i:04d}.png'))
                else:
                    crop.save(os.path.join(crop_path, f'{patient}_s_{i:04d}_n.png'))
        offset = aorta_end - branch_start
        start, end = branch_start + int(0.1*offset), aorta_end - int(0.2*offset)
        for i in range(start-3, end+3):
            try:
                img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
                img = np.array(img)
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                if start <= i < end:
                    crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png'))
                else:
                    crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}_n.png'))
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)
            
epl1 = []
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative', epl1)

******Processing baimeizhu-27-37-54-136******
******Processing baixiaoxu-22-31-52-147******
******Processing baojinjia-17-21-35-83******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：baojinjia-17-21-35-83_0018.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：baojinjia-17-21-35-83_0019.txt


******Processing baoxialin-23-34-57-158******
******Processing baoyanzu-15-20-31-87******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：baoyanzu-15-20-31-87_0017.txt


******Processing baoyanzu-21-25-42-93******


...

Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhuxinnan-18-26-49-129_0024.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhuxinnan-18-26-49-129_0025.txt


******Processing zhuyanjie-20-25-35-89******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhuyanjie-20-25-35-89_0022.txt


******Processing zoudeling-11-15-30-84******


Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zoudeling-11-15-30-84_0012.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "/tmp/ipykernel_37540/2245311451.py", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zoudeling-11-15-30-84_0013.txt
Traceback (most recent call last):
  File "/tmp/ipykernel_37540/2245311451.py", line 44, in 

In [11]:
print(len(epl1))
print(epl1)

347
['baojinjia-17-21-35-83', 'baoyanzu-15-20-31-87', 'baoyanzu-21-25-42-93', 'bianyinqiao-31-38-59-137', 'bianzhaorong-17-22-38-92', 'caijinfei-34-42-51-155', 'caijingnan1-41-50-73-184', 'caishiping-18-24-34-85', 'caiyouzhe-14-18-32-83', 'caoyuxia-16-21-35-87', 'caoyuying-18-22-33-79', 'chenbin-20-26-35-85', 'chenchong-22-26-36-86', 'chendonghong-12-17-27-75', 'chengenye-13-18-28-79', 'chengshaoping-19-28-45-132', 'chengyahui-17-22-29-82', 'chengzhiqiang-33-41-67-159', 'chenjian-26-30-44-89', 'chenjianhua-14-17-32-80', 'chenjianping-15-22-41-91', 'chenjingjing-38-47-67-184', 'chenjinmei-12-17-34-80', 'chenjunbing-22-29-34-89', 'chenlidong-18-23-32-88', 'chenliuer-23-32-43-140', 'chenmaojin-14-20-31-90', 'chenqiuying-13-17-30-78', 'chenshigen-19-25-34-83', 'chenweiqian-22-31-45-131', 'chenxiufen-17-21-34-82', 'chenyanli-20-23-34-80', 'chenyinfen-18-22-38-81', 'chenyonghu-24-32-48-141', 'daicaihu-22-27-34-85', 'denghuibin-15-19-35-84', 'dingbinghai-20-24-45-95', 'dongxingzhen-9-13-23-77

## 3.固定训练集和测试集

### 3.1 阳性数据

In [28]:
patient_roots = ['/nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau']
patient_list = []
for patient_root in patient_roots:
    for patient in sorted(os.listdir(patient_root)):
        if os.path.isdir(os.path.join(patient_root, patient)):
            patient_list.append(os.path.join(patient_root, patient))

In [29]:
random.seed(7987)
val_patient_list = random.sample(patient_list, int(len(patient_list)*0.2))
train_patient_list = [p for p in patient_list if p not in val_patient_list]#list(set(patient_list) - set(val_patient_list))
train_patient_list.sort()
val_patient_list.sort()
print(len(train_patient_list), len(val_patient_list), len(train_patient_list)+len(val_patient_list))

63 15 78


In [30]:
def move_to_classify(patient_list, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for patient in patient_list:
        print(f'****Processing {patient}****')
        for file in os.listdir(os.path.join(patient, 'crops')):
            shutil.copy(os.path.join(patient, 'crops', file), output_path)
            
move_to_classify(train_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/center/train/3/')
move_to_classify(val_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/center/val/3/')

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/bairuixin-J-29-46****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/baochangmu-J-18-77****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/baodezhong-J-41-44****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/chenbingrong-J-61-96****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/chenmazhang-J-18-22****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/chensimeng-J-31-33****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/dongyoufa-J-48-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/fanggaoshen-J-82-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/fangpingan-J-29-34****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/feiliangming-J-23-41****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/gaohuarong-J-51-109****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/pau/gaozhihua-J-82-94****


### 3.2阴性数据

In [13]:
patient_roots = ['/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/']
patient_list = []
for patient_root in patient_roots:
    for patient in sorted(os.listdir(patient_root)):
        if os.path.isdir(os.path.join(patient_root, patient)):
            patient_list.append(os.path.join(patient_root, patient))

In [14]:
random.seed(7888)
val_patient_list = random.sample(patient_list, int(len(patient_list)*0.2))
train_patient_list = [p for p in patient_list if p not in val_patient_list]#list(set(patient_list) - set(val_patient_list))
train_patient_list.sort()
val_patient_list.sort()
print(len(train_patient_list), len(val_patient_list), len(train_patient_list)+len(val_patient_list))

379 94 473


In [15]:
def move_to_classify(patient_list, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for patient in patient_list:
        print(f'****Processing {patient}****')
        for file in os.listdir(os.path.join(patient, 'crops')):
            shutil.copy(os.path.join(patient, 'crops', file), output_path)
            
move_to_classify(train_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/center/train/0/')
move_to_classify(val_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/center/val/0/')

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baixiaoxu-22-31-52-147****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baojinjia-17-21-35-83****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baoxialin-23-34-57-158****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baoyanzu-15-20-31-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baoyanzu-21-25-42-93****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/bianzhaorong-17-22-38-92****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caijinfei-34-42-51-155****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caijingnan1-41-50-73-184****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caiqishu-34-43-70-176****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caishiping-18-24-34-85****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caiyouzhe-14-18-3

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/huyongxian-18-25-38-88****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/huyunhua-15-20-28-78****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/huzaoheng-16-23-34-67****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/huzhengjin-19-27-48-150****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/jiangchengsan-25-34-50-133****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/jiangjianrong-16-21-31-82****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/jiangjiaqi-23-30-39-140****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/jiangjinfeng-26-39-58-152****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/jiangshengfang-19-23-34-79****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/jiangyuxiang-15-19-37-83****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/puchenhu-18-21-33-89****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/puguanghui-12-18-25-74****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/qianaqin-18-26-52-141****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/qihuazhong-16-20-29-82****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/qiuwei-31-39-53-138****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/qizhikun-29-37-57-130****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/qubo-14-19-29-82****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/quzhuming-31-41-69-192****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/renjiali-34-42-52-138****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/renlaifa-19-30-53-153****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/shanziliang-13-19-30-90****
**

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiaxinghua-17-22-33-89****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiemeiying-21-29-41-127****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xieshangyong-14-20-27-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiezhaoqin-17-21-29-81****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiyongmin-25-35-62-162****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xucaizhen-27-37-51-131****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xudingguo-21-25-38-89****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xuguangtai-16-21-36-91****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xurihua-26-34-61-145****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xushengwu-31-41-60-150****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xuxuee-22-30-45

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouhaibin-30-39-74-185****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhoulijun-18-23-35-93****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhoushijun-30-37-64-145****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouweiqiang-19-24-42-90****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouwenming-40-48-67-147****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouxianhang-13-17-26-78****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouxinya-24-30-40-120****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouyixiao-22-28-40-83****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouyongjin-17-21-35-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouyongyan-21-33-51-136****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhouz

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/wangyunxian-18-27-39-134****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/wenshuijuan-30-36-66-145****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiangzhaochong-15-20-31-78****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiasuifu-12-18-30-84****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiegenjun-21-26-34-93****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xinyongsheng-13-18-26-86****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/yanchunhong-36-45-64-155****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/yangqingyue-31-39-50-128****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/yangtangyong-15-19-35-85****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/yangxiaohui-20-28-39-125****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/

## 4.检查并统计crop图像尺寸

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from numpy.lib.function_base import median

paths = [f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/0/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/0/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/1/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/1/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/2/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/2/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/3/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/3/',
        ]

size_list = []

count = 0
for path in paths:
    path_count = 0
    for f in os.listdir(path):
        path_count += 1
        img = Image.open(os.path.join(path, f))
        size_list.append(img.height)
        if img.height != img.width:
            print(f'{os.path.join(path, f)}: height:{img.height}, width:{img.width}')
            count += 1
    print(f'{path}: {path_count}')

size_list.sort()

sum = 0
hash_list = {}
for size in size_list:
    sum += size
    hash_list[size] = hash_list.get(size, 0) + 1

avg = sum / len(size_list)

key = len(size_list) // 2
med = (size_list[key] + size_list[-key-1]) / 2

v_list = list(hash_list.items())
v_list.sort(key=lambda x: x[1], reverse=True)
mode = v_list[0][0]

print(f'avg:{avg}, median:{med}, mode:{mode}({v_list[0][1]}), h_w_not_equal:{count}')

In [ ]:
interval = 3
bin_num = (max(size_list)-min(size_list)) // interval
bins = [i for i in range(min(size_list), max(size_list)+interval, interval)]
fig = plt.figure(figsize=(30, 8), dpi=160)
plt.hist(size_list, bins=bins)
plt.xticks(ticks=bins)
plt.show()

In [ ]:
print(max(size_list))

In [ ]:
img_path = '/nfs3-p1/zsxm/dataset/aorta_classify_ct_-100_500/val/0/'

for img in os.listdir(img_path):
    nl = img.split('-')
    if len(nl) != 5:
        print(1, img)
    if '0'<= nl[0][-1] <='9':
        print(2, img)

In [ ]:
old_img_list = []
new_img_list = []

old_list = ['/nfs3-p2/zsxm/dataset/aorta_classify_ct_-100_500/negative_deleted/train_0_backup',
            '/nfs3-p2/zsxm/dataset/aorta_classify_ct_-100_500/negative_deleted/val_0_backup']
new_list = ['/nfs3-p2/zsxm/dataset/aorta_classify_ct_-100_500/train/0', 
            '/nfs3-p2/zsxm/dataset/aorta_classify_ct_-100_500/val/0']
for path in old_list:
    for img in os.listdir(path):
        old_img_list.append(img)
for path in new_list:
    for img in os.listdir(path):
        new_img_list.append(img)
print(len(old_img_list), len(new_img_list))

In [ ]:
for img in new_img_list:
    if img not in old_img_list:
        print(img)

In [ ]:
for img in old_img_list:
    if img not in new_img_list:
        print(img)